In [2]:
# %pip install torch
# %pip install pandas
# %pip install nltk
# %pip install rouge-score==0.0.4
# %pip uninstall -y transformers
# %pip install transformers
# %pip freeze
# %pip install --upgrade pip setuptools
# %pip install --no-cache-dir transformers==4.10.0
# %pip install --index-url=https://pypi.python.org/simple/transformers==4.10.0
# %pip install git+https://github.com/huggingface/transformers
# %pip install tokenizers --no-index --find-links https://download.pytorch.org/whl/torch_stable.html

  Obtaining dependency information for torch from https://files.pythonhosted.org/packages/2c/df/5810707da6f2fd4be57f0cc417987c0fa16a2eecf0b1b71f82ea555dc619/torch-2.2.1-cp311-cp311-manylinux1_x86_64.whl.metadata
  Using cached torch-2.2.1-cp311-cp311-manylinux1_x86_64.whl.metadata (26 kB)
  Obtaining dependency information for typing-extensions>=4.8.0 from https://files.pythonhosted.org/packages/f9/de/dc04a3ea60b22624b51c703a84bbe0184abcd1d0b9bc8074b5d6b7ab90bb/typing_extensions-4.10.0-py3-none-any.whl.metadata
  Using cached typing_extensions-4.10.0-py3-none-any.whl.metadata (3.0 kB)
  Obtaining dependency information for nvidia-cuda-nvrtc-cu12==12.1.105 from https://files.pythonhosted.org/packages/b6/9f/c64c03f49d6fbc56196664d05dba14e3a561038a81a638eeb47f4d4cfd48/nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Obtaining dependency information for nvidia-cuda-runti

In [16]:
### Imports

In [17]:
import os
import numpy as np
import pandas as pd
import json
import warnings
import logging
import gc
import random
import math
import re
import ast
from tqdm import tqdm
from typing import Optional
from datetime import datetime

import nltk
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.meteor_score import meteor_score
from rouge_score.rouge_scorer import RougeScorer

warnings.filterwarnings("ignore")

import torch
from torch.utils.data import DataLoader, TensorDataset


### Config

In [18]:
TARGET_COLUMN = 'code_mixed_explanation'
TEXT_INPUT_PATH = '../Data/Text/'
ACOUSTIC_INPUT_PATH = '../Data/Audio/'
VISUAL_INPUT_PATH = '../Data/Video/'

MODEL_OUTPUT_DIR = '../models/MAF-TAV-BART/'
RESULT_OUTPUT_DIR = '../results/MAF-TAV-BART/'

LOWERCASE_UTTERANCES = False
UNFOLDED_DIALOGUE = True

if UNFOLDED_DIALOGUE:
    SOURCE_COLUMN = 'dialogue'
else:
    SOURCE_COLUMN_1 = 'target'
    SOURCE_COLUMN_2 = 'context'
    


SOURCE_MAX_LEN = 480
TARGET_MAX_LEN = 50
MAX_UTTERANCES = 25

ACOUSTIC_DIM = 154
ACOUSTIC_MAX_LEN = 600

VISUAL_DIM = 2048
VISUAL_MAX_LEN = 96 

BATCH_SIZE = 4
MAX_EPOCHS = 60

BASE_LEARNING_RATE = 5e-6
NEW_LEARNING_RATE = 5e-5
WEIGHT_DECAY = 1e-4

NUM_BEAMS = 5
EARLY_STOPPING = True
NO_REPEAT_NGRAM_SIZE = 3

EARLY_STOPPING_THRESHOLD = 5


In [19]:
def set_random_seed(seed: int):
    """
    Helper function to seed experiment for reproducibility.
    If -1 is provided as seed, experiment uses random seed from 0~9999
    Args:
        seed (int): integer to be used as seed, use -1 to randomly seed experiment
    """
    print("Seed: {}".format(seed))


    # These lines below ensure determinism by turning off some optimizations
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.enabled = False
    torch.backends.cudnn.deterministic = True

    if seed == -1:
        seed = random.randint(0, 9999)

    # We set all seeds in libraries to our seed
    
    random.seed(seed) #python built-in
    os.environ["PYTHONHASHSEED"] = str(seed) #seed for hash based ops in python
    np.random.seed(seed) #numpy random gen
    torch.manual_seed(seed) # torch CPU random generator
    torch.cuda.manual_seed(seed) # current gpu rand gen
    torch.cuda.manual_seed_all(seed) # all gpus rand gen


In [20]:
if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
    print("Using GPU")
else:
    DEVICE = torch.device("cpu")
    print("Using CPU")


Using GPU


In [21]:
set_random_seed(42)

Seed: 42


### Model

In [1]:
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.checkpoint
from torch.nn import CrossEntropyLoss, MSELoss

from typing import Any, Callable, Dict, Iterable, List, Optional, Tuple, Union

from transformers.modeling_utils import PreTrainedModel, unwrap_model

from transformers import (
    BartTokenizerFast,
    AdamW
)

from transformers.models.bart.configuration_bart import BartConfig

# from transformers.models.bart.modeling_bart import (
#     BartPretrainedModel,
#     BartDecoder,
#     BartLearnedPositionalEmbedding,
#     BartEncoderLayer,
#     shift_tokens_right,
#     _make_causal_mask,
#     _expand_mask 
# )


from transformers.modeling_outputs import (
    BaseModelOutput,
    Seq2SeqLMOutput,
    Seq2SeqModelOutput
)


# from transformer_encoder import TransformerEncoder


### Training setup

In [ ]:
gen_kwargs = {
        'num_beams': NUM_BEAMS,
        'max_length': TARGET_MAX_LEN,
        'early_stopping': EARLY_STOPPING,
        'no_repeat_ngram_size': NO_REPEAT_NGRAM_SIZE
    }
    
    train(model=MODEL,
          tokenizer=TOKENIZER,
          train_data_loader=train_dataset,
          val_data_loader=val_dataset,
          test_data_loader=test_dataset,
          base_learning_rate=BASE_LEARNING_RATE,
          new_learning_rate=NEW_LEARNING_RATE,
          weight_decay=WEIGHT_DECAY,
          **gen_kwargs)
    
    print("Model Trained!")